In [ ]:
import os
import keras as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [ ]:
file = 'dataset_1'

train_datagen = K.preprocessing.image.ImageDataGenerator()

validation_datagen = K.preprocessing.image.ImageDataGenerator()

test_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(file, 'train'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        )

validation_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'validation'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

test_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'test'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

In [ ]:
def runGenerator(generator):
    X = []
    y = []
    for i in range(len(generator)):
        X_ex, y_ex = generator.next()
        X.append(X_ex)
        y.append(y_ex)
    return X, y

def F1ScoreCalculator(test_generatorm, model):
    X_test = []
    y_test = []
    y_pred = []
    for i in range(len(test_generator)):
        X_ex, y_ex = test_generator.next()
        X_test.append(X_ex)
        y_test.append(y_ex.argmax(axis=-1))
        y_pred.append(model.predict(X_ex).argmax(axis=-1))

    return f1_score(y_test, y_pred, average='micro')

In [ ]:
def Create_model(kernel_sizes=5, neurons_bk1=20, neurons_bk2=50, learn_rate=0.001):
    model = Sequential()

    model.add(Conv2D(neurons_bk1, kernel_size=kernel_sizes, padding="same", input_shape=(50, 50, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(neurons_bk2, kernel_size=kernel_sizes, border_mode="same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation('relu'))

    model.add(Dense(28))
    model.add(Activation("softmax"))
    
    model.summary()
    model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])
    return model

In [ ]:
early_stop = K.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='max')

In [ ]:
param_grid = dict(
    kernel_sizes=[5], 
#     neurons_bk1=[10, 20, 30], 
#     neurons_bk2=[50, 100],
#     learn_rate=[0.01, 0.005]
)

In [ ]:
INPUT_SHAPE = (50, 50, 1)
NB_EPOCH = 10
BATCH_SIZE = 32
VERBOSE = 1
OPTIMIZER = SGD()
CLASSES = 28
STEPS = (int)(train_generator.n/BATCH_SIZE)

In [ ]:
model = KerasClassifier(build_fn=Create_model, 
                        epochs=NB_EPOCH, 
                        batch_size=BATCH_SIZE, 
                        verbose=VERBOSE
                       )

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)


In [ ]:
X_train, y_train = runGenerator(train_generator)

In [ ]:
history = grid.fit(X_train, y_train)

In [ ]:
print ("F1-Score = {}".format(F1ScoreCalculator(test_generator, grid)))

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model = LeNetLeakyRelu.build(input_shape=INPUT_SHAPE, classes=CLASSES)

model.summary()

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS,
        epochs=NB_EPOCH,
#         verbose=VERBOSE,
        validation_data=validation_generator,
#         validation_steps=STEPS,
        callbacks = [early_stop]
)

In [ ]:
# pred = model.predict_generator(test_generator)
X_test = []
y_test = []
y_pred = []
for i in range(len(test_generator)):
    X_ex, y_ex = test_generator.next()
    X_test.append(X_ex)
    y_test.append(y_ex.argmax(axis=-1))
    y_pred.append(model.predict(X_ex).argmax(axis=-1))
    
val_f1_score = f1_score(y_test, y_pred, average='micro')
val_f1_score

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()